In [66]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import time

In [67]:
bvid = "BV15iBuY9E87"  # 替换为目标视频的BVID（URL中的BV号）

# 获取视频信息（包含cid和aid）
url_info = f"https://api.bilibili.com/x/web-interface/view?bvid={bvid}"
response = requests.get(url_info, headers={"User-Agent": "Mozilla/5.0"})
data = response.json()
cid = data['data']['cid']  # 弹幕和字幕需要cid
oid = data['data']['aid']  # 评论需要oid（即aid）

In [68]:
bvid = "BV15iBuY9E87"  # 替换为你的BVID

# 请求视频信息API
url = f"https://api.bilibili.com/x/web-interface/view?bvid={bvid}"
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
data = response.json()

# 提取CID（默认获取第一个分P的CID）
cid = data['data']['pages'][0]['cid']
print("CID:", cid)

CID: 27327596302


In [69]:
oid = data['data']['aid']
print("OID:", oid)

OID: 113646596263696


In [70]:
# 请求弹幕API
danmaku_url = f"https://api.bilibili.com/x/v1/dm/list.so?oid={cid}"
danmaku_resp = requests.get(danmaku_url, headers={"User-Agent": "Mozilla/5.0"})
danmaku_resp.encoding = 'utf-8'

# 解析XML
soup = BeautifulSoup(danmaku_resp.text, 'xml')
danmus = soup.find_all('d')
danmu_list = [danmu.text for danmu in danmus]

# 保存为DataFrame
df_danmu = pd.DataFrame(danmu_list, columns=['弹幕内容'])

In [71]:
subtitle_url = f"https://api.bilibili.com/x/player/v2?cid={cid}&bvid={bvid}"
resp = requests.get(subtitle_url, headers={"User-Agent": "Mozilla/5.0"})
data = resp.json()

subtitle_list = []
if 'subtitle' in data['data'] and data['data']['subtitle']['subtitles']:
    subtitle_url = data['data']['subtitle']['subtitles'][0]['subtitle_url']
    subtitle_resp = requests.get(f"https:{subtitle_url}")
    subtitle_data = subtitle_resp.json()
    for item in subtitle_data['body']:
        subtitle_list.append({"时间": item['from'], "字幕内容": item['content']})

df_subtitle = pd.DataFrame(subtitle_list)

In [72]:
with pd.ExcelWriter('bilibili_data.xlsx', engine='openpyxl') as writer:
    df_danmu.to_excel(writer, sheet_name='弹幕', index=False)
    if not df_subtitle.empty:
        df_subtitle.to_excel(writer, sheet_name='字幕', index=False)

In [73]:
def get_all_comments(oid):
    comments = []
    page = 1  # 主评论分页起始
    
    while True:
        # 请求主评论API
        url = f"https://api.bilibili.com/x/v2/reply/main?jsonp=jsonp&next={page}&type=1&oid={oid}"
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(url, headers=headers)
        data = response.json()
        
        # 检查是否无数据
        if not data.get('data') or not data['data'].get('replies'):
            break
        
        # 处理主评论
        for reply in data['data']['replies']:
            # 添加主评论
            comments.append({
                "类型": "主评论",
                "评论ID": reply['rpid'],
                "用户": reply['member']['uname'],
                "内容": reply['content']['message'],
                "点赞数": reply['like'],
                "回复数": reply['count']
            })
            
            # 如果有次级评论，抓取全部
            if reply['count'] > 0:
                sub_comments = get_sub_comments(oid, reply['rpid'])
                comments.extend(sub_comments)
        
        # 检查是否还有下一页
        if data['data']['cursor']['is_end']:
            break
        
        page += 1
        time.sleep(1)  # 防止请求过快
        
    return comments

def get_sub_comments(oid, root_id):
    sub_comments = []
    sub_page = 1  # 次级评论分页起始
    
    while True:
        url = f"https://api.bilibili.com/x/v2/reply/reply?jsonp=jsonp&pn={sub_page}&type=1&oid={oid}&root={root_id}"
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(url, headers=headers)
        data = response.json()
        
        if not data.get('data') or not data['data'].get('replies'):
            break
        
        for reply in data['data']['replies']:
            sub_comments.append({
                "类型": "次级评论",
                "评论ID": reply['rpid'],
                "用户": reply['member']['uname'],
                "内容": reply['content']['message'],
                "点赞数": reply['like'],
                "回复对象": f"{reply['parent']}@{reply['root']}"  # 被回复的评论ID
            })
        
        # 检查是否还有下一页
        if data['data']['page']['num'] * data['data']['page']['size'] >= data['data']['page']['count']:
            break
        
        sub_page += 1
        time.sleep(0.5)
    
    return sub_comments

# 使用示例
oid = 113646596263696  # 替换为实际OID
all_comments = get_all_comments(oid)
df_comments = pd.DataFrame(all_comments)
df_comments.to_excel("bilibili_comments_4.xlsx", index=False)